# CNN (U-NET Below)

In [5]:
import keras
import matplotlib.image as mpimg
import numpy
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import tensorflow as tf
import matplotlib.colors as colors

Using TensorFlow backend.
/Users/romain/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [244]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - numpy.min(img)
    rimg = (rimg / numpy.max(rimg) * PIXEL_DEPTH).round().astype(numpy.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def extract_labels(filename, num_images, stride = IMG_PATCH_SIZE, verbose=1):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            if verbose:
                print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE, stride) for i in range(num_images)]
    data = numpy.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = numpy.asarray([value_to_class(numpy.mean(data[i])) for i in range(len(data))])
    # Convert to dense 1-hot representation.
    return labels.astype(numpy.float32)

def img_crop(im, w, h, stride):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight-numpy.mod(stride,h),stride):
        for j in range(0,imgwidth-numpy.mod(stride,w),stride):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
    df = numpy.sum(v)
    if df > foreground_threshold:
        return [0, 1]
    else:
        return [1, 0]
        
def extract_data(filename, num_images,stride = IMG_PATCH_SIZE, verbose=1):
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            if verbose:
                print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE, stride) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]
    
    return numpy.asarray(data)

In [226]:
def extract_images(filename, num_images,verbose=1):
    
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            if verbose:
                print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(imgs)    
    return numpy.asarray(imgs)

def extract_groundtruth(filename, num_images, verbose=1):
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            if verbose:
                print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    return numpy.asarray(gt_imgs).astype(numpy.float32)

def 

In [224]:
def add_rotations(patches,labels):
    new_patches = [numpy.rot90(patches[j],i) for i in range(4) for j in range(len(patches))]
    new_labels = numpy.repeat(labels,4,axis=0)
    return numpy.asarray(new_patches),numpy.asarray(new_labels)

def rgb_to_hsv(patches):
    new_patches = [colors.rgb_to_hsv(patches[i]) for i in range(len(patches))]
    return numpy.asarray(new_patches)

def create_bagging_sets(train_data,train_labels,ratio,n):

    numpy.random.seed()
    
    l = numpy.size(train_data,0)
    batch_size = int(numpy.floor(ratio*l))
    indices = numpy.random.randint(l,size=(n,batch_size))
    
    data_sets = [train_data[indices[i]] for i in range(n)]
    label_sets = [train_labels[indices[i]] for i in range(n)]
    
    
    return numpy.asarray(data_sets), numpy.asarray(label_sets)

In [256]:
from sklearn.model_selection import train_test_split


PIXEL_DEPTH = 255
TRAINING_SIZE = 100
BATCH_SIZE = 16 # 64
NUM_EPOCHS = 5
BAGGING_SIZE = 3
BAGGING_RATIO = 0.5
TRAINING_STRIDE = 8

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

def prepare_data(verbose=1,stride=TRAINING_STRIDE):

    root_dir = "training/"
    image_dir = root_dir + "images/"
    gt_dir = root_dir + "groundtruth/"
    
    if verbose:
        print("Extracting data...")
    train_data = extract_data(image_dir, TRAINING_SIZE, stride=stride, verbose=0)
    
    if verbose:
        print('Extracting labels...')
    train_labels = extract_labels(gt_dir, TRAINING_SIZE, stride=stride, verbose=0)
    
    if verbose:
        print('Changing color representation')
    # Use HSV instead of RGB
    train_data  = rgb_to_hsv(train_data)

    # Add rotations to make model rotation-invariant
    #train_data, train_labels = add_rotations(train_data,train_labels)

    if verbose:
        print('Spliting data for cross-validation')
    # Seperate data for cross-validation
    return train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [252]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
    
def create_model(verbose=0):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(16, 16,3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))

    sgd = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    if verbose:
        print(model.summary())
    
    return model

def train_models(X_train, X_test, y_train, y_test, bagging_size=3, bagging_ratio=0.4, epochs=5, batch_size=16,verbose=1):
    
    models = []
    
    if verbose:
        print('Creating bagging data sets...')
        
    data_sets ,label_sets = create_bagging_sets(X_train, y_train, bagging_ratio, bagging_size)
    
    for i in range(bagging_size):
        
        if verbose:
            print('\n Training model n°%d' % int(i+1))
            
        model = create_model(verbose=0)
        model.fit(data_sets[i],label_sets[i],validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose = verbose)
        models.append(model)
    
    return models

def predict_with_bagging(models, data, labels=[], soft_voting=False, verbose=1):
    
    if verbose:
        print('\nMaking predictions...')
        
    n = len(models)    
    preds = numpy.asarray([models[i].predict(data, verbose=0) for i in range(n)])
    
    predictions = numpy.mean(preds,axis=0)
    hard_predictions = numpy.zeros(predictions.shape)
    
    indices = numpy.arange(len(predictions)).reshape((len(predictions),1))
    
    if soft_voting:
        values = numpy.argmax(predictions,1).reshape((len(predictions),1))
    else:
        values = numpy.rint(numpy.mean(numpy.argmax(preds,2),0)).astype(int).reshape((len(predictions),1))

    hard_predictions[indices,values] = 1
    
    if len(labels):
        acc = numpy.count_nonzero(numpy.subtract(hard_predictions,labels))/(2*len(labels))
        print("Prediction accuracy: %.2f%%" % ((1-acc)*100))
        
    return predictions , hard_predictions
        

In [255]:
X_train, X_test, y_train, y_test = prepare_data()

models = train_models(X_train, X_test, y_train, y_test, 
                      bagging_size=BAGGING_SIZE, bagging_ratio=BAGGING_RATIO, 
                      epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

preds,hard_preds = predict_with_bagging(models,X_test,y_test,soft_voting = False)

Creating bagging data sets...

 Training model n°1
Train on 96040 samples, validate on 48020 samples
Epoch 1/5
96040/96040 [==============================] - 152s 2ms/step - loss: 0.4501 - acc: 0.7834 - val_loss: 0.4002 - val_acc: 0.8095
Epoch 2/5
96040/96040 [==============================] - 175s 2ms/step - loss: 0.3943 - acc: 0.8129 - val_loss: 0.3940 - val_acc: 0.8028
Epoch 3/5
96040/96040 [==============================] - 176s 2ms/step - loss: 0.3711 - acc: 0.8254 - val_loss: 0.3605 - val_acc: 0.8330
Epoch 4/5
96040/96040 [==============================] - 146s 2ms/step - loss: 0.3560 - acc: 0.8337 - val_loss: 0.3480 - val_acc: 0.8390
Epoch 5/5
96040/96040 [==============================] - 161s 2ms/step - loss: 0.3432 - acc: 0.8400 - val_loss: 0.3479 - val_acc: 0.8391

 Training model n°2
Train on 96040 samples, validate on 48020 samples
Epoch 1/5
96040/96040 [==============================] - 179s 2ms/step - loss: 0.4513 - acc: 0.7837 - val_loss: 0.3951 - val_acc: 0.8134
Epoch 

# U-Net

In [9]:
import keras
import matplotlib.image as mpimg
import numpy
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import tensorflow as tf

In [10]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - numpy.min(img)
    rimg = (rimg / numpy.max(rimg) * PIXEL_DEPTH).round().astype(numpy.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = numpy.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = numpy.asarray([value_to_class(numpy.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(numpy.float32)

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
    df = numpy.sum(v)
    if df > foreground_threshold:
        return [0, 1]
    else:
        return [1, 0]
        
def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return numpy.asarray(data)


In [11]:
from sklearn.model_selection import train_test_split


NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 20
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16 # 64
NUM_EPOCHS = 5
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 1000
IMG_PATCH_SIZE = 16

root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

train_data = extract_data(image_dir, TRAINING_SIZE)
train_labels = extract_labels(gt_dir, TRAINING_SIZE)


X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Loading training/images/satImage_001.png
Loading training/images/satImage_002.png
Loading training/images/satImage_003.png
Loading training/images/satImage_004.png
Loading training/images/satImage_005.png
Loading training/images/satImage_006.png
Loading training/images/satImage_007.png
Loading training/images/satImage_008.png
Loading training/images/satImage_009.png
Loading training/images/satImage_010.png
Loading training/images/satImage_011.png
Loading training/images/satImage_012.png
Loading training/images/satImage_013.png
Loading training/images/satImage_014.png
Loading training/images/satImage_015.png
Loading training/images/satImage_016.png
Loading training/images/satImage_017.png
Loading training/images/satImage_018.png
Loading training/images/satImage_019.png
Loading training/images/satImage_020.png
Loading training/groundtruth/satImage_001.png
Loading training/groundtruth/satImage_002.png
Loading training/groundtruth/satImage_003.png
Loading training/groundtruth/satImage_004.

In [12]:
import os 
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate, Dense, Flatten
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


input_x = Input(shape=(16, 16,3), name='input_x') 
conv1 = Conv2D(64, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_x)
print("conv1 shape:",conv1.shape)
conv1 = Conv2D(64, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
print("conv1 shape:",conv1.shape)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
print("pool1 shape:",pool1.shape)

conv2 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
print("conv2 shape:",conv2.shape)
conv2 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
print("conv2 shape:",conv2.shape)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
print("pool2 shape:",pool2.shape)

conv3 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
print("conv3 shape:",conv3.shape)
conv3 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
print("conv3 shape:",conv3.shape)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
print("pool3 shape:",pool3.shape)

conv4 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
#merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
merge6 = concatenate([drop4,up6],axis = 3)
conv6 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
#merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
#merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
#merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
#print(conv10.shape)


res = Dense(2, activation='sigmoid')(Flatten()(conv9))

print(res.shape)
model = Model(inputs = input_x, outputs = res) #conv10

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

conv1 shape: (?, 16, 16, 64)
conv1 shape: (?, 16, 16, 64)
pool1 shape: (?, 8, 8, 64)
conv2 shape: (?, 8, 8, 128)
conv2 shape: (?, 8, 8, 128)
pool2 shape: (?, 4, 4, 128)
conv3 shape: (?, 4, 4, 256)
conv3 shape: (?, 4, 4, 256)
pool3 shape: (?, 2, 2, 256)
(?, 2)


In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=10, verbose=1,validation_split=0.2, shuffle=True)
#model.fit(X_train, y_train, batch_size=16, epochs=10, verbose=1,validation_data=(X_test, y_test), shuffle=True)
#print('predict test data')
#y_test = model.predict(X_test, batch_size=1, verbose=1)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Prediction

In [ ]:
def label_to_img(imgwidth, imgheight, w, h, labels):
    array_labels = numpy.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if labels[idx][0] > 0.5:
                l = 0
            else:
                l = 1
            array_labels[j:j+w, i:i+h] = l
            idx = idx + 1
    return array_labels

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def bw(gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = gt_img #np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        cimg = gt_img_3c
    return cimg

    # Get prediction for given input image 
def get_prediction(img):
    data = numpy.asarray(img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    output_prediction = model.predict(data)
    img_prediction = label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

    # Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(filename, image_idx):
    
    imageid = "test_" + str(image_idx)
    image_filename = filename + imageid + ".png"
    img = mpimg.imread(image_filename)
    
    img_prediction = get_prediction(img)
    #cimg = concatenate_images(img, img_prediction)
    #cimg = Image.fromarray(bw(img_prediction))

    #return cimg
    return img_prediction

submission_filename = 'submission.csv'
image_filenames = []

for i in range(1, TRAINING_SIZE+1):
    'test_set_images/test_'
    test_data_filename = 'test_set_images/test_' + str(i) + '/'
    pimg = get_prediction_with_groundtruth(test_data_filename, i)
    image_filename = 'test_set_images/pred_' + str(i) + '.png'
    Image.fromarray(bw(pimg)).save(image_filename)
    print(image_filename)
    image_filenames.append(image_filename)
